# ACC RAG Chain LLM

In [76]:
import os
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader

from src.fs import read_text_dir

## Read ACR Conditions

In [35]:
docs_raw = read_text_dir("data/conditions")
docs_text = "\n\n---\n\n".join(docs_raw.values())


## Indexing

### Load ACR Data

In [4]:
loader = DirectoryLoader("data/conditions", glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()
docs

[Document(metadata={'source': 'data/conditions/Mesenteric_Ischemia.md'}, page_content='# Imaging of Mesenteric Ischemia  \n\nAmerican College of Radiology: ACR Appropriateness Criteria®  \n\n### Variant 1: **Suspected acute mesenteric ischemia. Initial imaging.**\n\n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| CTA abdomen and pelvis with IV contrast    | Usually Appropriate            | ⚫⚫⚫⚫                  |\n| CT abdomen and pelvis with IV contrast     | May Be Appropriate             | ⚫⚫⚫                    |\n| Arteriography abdomen                      | May Be Appropriate (Disagreement) | ⚫⚫⚫               |\n| MRA abdomen and pelvis without and with IV contrast | May Be Appropriate (Disagreement) | O               |\n| Radiography abdomen                        | May Be Appropriate             | ⚫⚫         

In [17]:
docs[0].page_content

'# Imaging of Mesenteric Ischemia  \n\nAmerican College of Radiology: ACR Appropriateness Criteria®  \n\n### Variant 1: **Suspected acute mesenteric ischemia. Initial imaging.**\n\n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| CTA abdomen and pelvis with IV contrast    | Usually Appropriate            | ⚫⚫⚫⚫                  |\n| CT abdomen and pelvis with IV contrast     | May Be Appropriate             | ⚫⚫⚫                    |\n| Arteriography abdomen                      | May Be Appropriate (Disagreement) | ⚫⚫⚫               |\n| MRA abdomen and pelvis without and with IV contrast | May Be Appropriate (Disagreement) | O               |\n| Radiography abdomen                        | May Be Appropriate             | ⚫⚫                     |\n| US duplex Doppler abdomen                  | May Be Appropriate      

### Split Documents

Split by markdown headings

In [53]:
# Determine levels of headings to be splitted
headers_to_split_on = [
    ("#", "Header 1"),
    # ("##", "Header 2"),
    # ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, 
                                               strip_headers = False)
docs_splited = markdown_splitter.split_text(docs_text)
# docs_splited = [markdown_splitter.split_text(doc.page_content) for doc in docs]

In [54]:
docs_splited

[Document(metadata={'Header 1': 'Abnormal Liver Function Tests'}, page_content='# Abnormal Liver Function Tests  \nAmerican College of Radiology: ACR Appropriateness Criteria®  \n### Variant 1: **Abnormal liver function tests. Hepatocellular predominance with mild aminotransferase increase. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| US abdomen                                 | Usually Appropriate            | O                        |\n| US duplex Doppler abdomen                  | Usually Appropriate            | O                        |\n| US shear wave elastography abdomen         | May Be Appropriate             | O                        |\n| MR elastography abdomen                    | May Be Appropriate             | O                        |\n| MRI abdomen without and with IV cont

### Create Vector Store & Retriever

In [ ]:
db = Chroma.from_documents(docs_splited, OpenAIEmbeddings(), persist_directory="db")

Test Similarity search

In [60]:
query = "Abnormal LFT"
docs = db.similarity_search(query, k = 2)
print(docs)

[Document(metadata={'Header 1': 'Abnormal Liver Function Tests', 'Header 3': 'Variant 3: **Abnormal liver function tests. Cholestatic predominance. Elevated alkaline phosphatase with or without elevated gamma-glutamyl transpeptidase. Initial imaging.**'}, page_content='### Variant 3: **Abnormal liver function tests. Cholestatic predominance. Elevated alkaline phosphatase with or without elevated gamma-glutamyl transpeptidase. Initial imaging.**  \n| Procedure                                  | Appropriateness Category       | Relative Radiation Level |\n|--------------------------------------------|--------------------------------|--------------------------|\n| US abdomen                                 | Usually Appropriate            | O                        |\n| MRI abdomen without and with IV contrast with MRCP | Usually Appropriate         | O                        |\n| CT abdomen and pelvis with IV contrast     | Usually Appropriate            | ⚫⚫⚫                    |\n| US 

In [75]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

## Prompt Template

In [95]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

system_prompt = """You are an clinical decision support assistance for radiologist. 
Your task is to determine the imaging appropriateness from the input patient case, according to the provided context from ACR Appropriateness Criteria.
"""

human_prompt = """
Case: 

{case}

Context: 
Use the provided context only to answer the question. If you don't know the answer, just say that you don't know.

{context}

Questions: Is imaging typically appropriate for this case? If so, please specify the most suitable imaging modality and whether a contrast agent is required. Exclude 'May Be Appropriate' and 'Usually Not Appropriate' as potential responses.
"""

prompt_template = ChatPromptTemplate([
    ("system", system_prompt),
    ("human", human_prompt)
])

print(prompt_template.invoke({"case": "Test Case", 
                              "context": "test context"}).to_string())



System: You are an clinical decision support assistance for radiologist. 
Your task is to determine the imaging appropriateness from the input patient case, according to the provided context from ACR Appropriateness Criteria.

Human: 
Case: 

Test Case

Context: 
Use the provided context only to answer the question. If you don't know the answer, just say that you don't know.

test context

Questions: Is imaging typically appropriate for this case? If so, please specify the most suitable imaging modality and whether a contrast agent is required. Exclude 'May Be Appropriate' and 'Usually Not Appropriate' as potential responses.



## Final Chain

In [92]:
llm = ChatOpenAI(model="gpt-4o-mini")
rag_chain = {"context": retriever, "case": RunnablePassthrough()} | prompt_template | llm

## Execute 🚀

### RLQ Abdominal Pain

In [93]:
case_1 = """
Patient age: 15 years old
Sex: Female
Chief Complaint: Right lower quadrant abdominal pain
The patient presents with a 2-day history of progressively worsening right lower quadrant abdominal pain. She reports loss of appetite but denies nausea or vomiting. No fever or diarrhea. Her last menstrual period was 2 weeks ago. She denies any history of similar pain episodes and has no significant past medical history. On physical examination, she appears uncomfortable. Vital signs show temperature 37.8°C, heart rate 92/min, blood pressure 110/70 mmHg. Abdominal examination reveals tenderness in the right lower quadrant with voluntary guarding but no rebound tenderness. McBurney's point is tender on palpation. Laboratory results show mild leukocytosis (WBC 12,500/µL) with neutrophil predominance.
"""

response_1 = rag_chain.invoke(case_1)
print(response_1.content)

Yes, imaging is typically appropriate for this case. The most suitable imaging modality is a CT abdomen and pelvis with IV contrast, as this is categorized as "Usually Appropriate" for suspected appendicitis in a patient with right lower quadrant pain, fever, and leukocytosis.


### Mesenteric Ischemia

In [97]:
case_2 = """
Patient age: 72 years old
Sex: Female
Chief Complaint: Severe abdominal pain and vomiting
The patient presents with sudden onset of severe, diffuse abdominal pain that started 6 hours ago, accompanied by multiple episodes of vomiting. She has a history of atrial fibrillation on warfarin, hypertension, and peripheral vascular disease. She reports the pain is out of proportion to physical examination findings. She also notes having similar but milder post-prandial abdominal pain and weight loss of 5 kg over the past 3 months, which made her eat less to avoid the pain. 
"""

response_2 = rag_chain.invoke(case_2)
print(response_2.content)

Yes, imaging is typically appropriate for this case. The most suitable imaging modality is a CT abdomen and pelvis with IV contrast, as it is classified as "Usually Appropriate" for suspected acute mesenteric ischemia, which aligns with the patient's presentation of severe abdominal pain and vomiting, particularly given her history of peripheral vascular disease.


### No Avilable Context

In [98]:
case_3 = """
Patient age: 30 years old 
Sex: Male. 
Chief complaint: Indeterminate palpable breast mass on the left side. 

A 2 cm hard mass is palpated in the left breast without nipple discharge or retraction, and no axillary lymphadenopathy. Mammography was indeterminant. There is no history of breast cancer in the family.
"""

response_3 = rag_chain.invoke(case_3)
print(response_3.content)

I don't know.
